In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import math
import time

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

import sys
sys.path.append('..')
from detector.input_pipeline import KeypointPipeline

# Build a graph

In [ ]:
tf.reset_default_graph()

files = [
    '/home/dan/datasets/COCO/multiposenet/train/shard-0010.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0020.tfrecords',
    '/home/dan/datasets/COCO/multiposenet/train/shard-0030.tfrecords',
]
params = {
    'batch_size': 16, 'image_height': 512, 'image_width': 512, 'min_dimension': 512
}

with tf.device('/cpu:0'):
    pipeline = KeypointPipeline(files, is_training=True, params=params)
    dataset = pipeline.dataset
    iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
    init = iterator.make_initializer(dataset)
    features, labels = iterator.get_next()
    features.update(labels)

# Show an image

In [ ]:
def plot_maps(image, heatmaps):

    h, w, _ = image.shape
    _, axes = plt.subplots(nrows=2, ncols=3, figsize=(int(w*3/50), int(h*2/50)))

    cmap = plt.cm.get_cmap('autumn')
    new_cmap = cmap(np.arange(cmap.N))
    new_cmap[:, -1] = np.sqrt(np.linspace(0, 1, cmap.N))  # set alpha
    cmap = ListedColormap(new_cmap)  # create new colormap

    for ax in axes.reshape(-1):
        ax.imshow(image)
        ax.axis('off')

    # ROW 1

    axes[0, 0].imshow(heatmaps[:, :, 0], cmap=cmap)
    axes[0, 0].set_title('noses')

    axes[0, 1].imshow(heatmaps[:, :, 11], cmap=cmap)
    axes[0, 1].set_title('left hips')

    axes[0, 2].imshow(heatmaps[:, :, 8], cmap=cmap)
    axes[0, 2].set_title('right elbows')
    
    # ROW 2

    axes[1, 0].imshow(heatmaps[:, :, 9], cmap=cmap)
    axes[1, 0].set_title('left wrists')

    axes[1, 1].imshow(heatmaps[:, :, 2], cmap=cmap)
    axes[1, 1].set_title('right eyes')

    axes[1, 2].imshow(heatmaps[:, :, 16], cmap=cmap)
    axes[1, 2].set_title('right ankles')

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    output = sess.run(features)

In [ ]:
i = 0
image = (255.0*output['images'][i]).astype('uint8')
heatmaps, masks = np.split(output['heatmaps_and_masks'][i], [17], axis=2)

h, w, _ = image.shape
heatmaps = cv2.resize(heatmaps, (w, h))
plot_maps(image, heatmaps)

In [ ]:
Image.fromarray(image)

In [ ]:
Image.fromarray(255*masks[:, :, 0].astype('uint8'))

In [ ]:
Image.fromarray(255*masks[:, :, 1].astype('uint8'))

# Measure speed

In [ ]:
times = []
with tf.Session() as sess:
    sess.run(init)
    for _ in range(105):
        start = time.perf_counter()
        output = sess.run(features)
        times.append(time.perf_counter() - start)

times = np.array(times[5:])
print(times.mean(), times.std())